Analysis of Estimation Error
=======================

**Requirements**:
- The [evo](https://github.com/MichaelGrupp/evo) Python package
    - Can be installed with `pip2 install evo -U --user`
    - Be sure that [this](https://github.com/MichaelGrupp/evo/pull/115) PR has been incorporated. If not, install `evo` from source so those changes are included.
- Python 2 kernel (for `import rosbag`)
- make sure you enabled interactive widgets via:
    ```
    jupyter nbextension enable --py widgetsnbextension
    ```
- rosbags with the `/vins_estimator/odometry` topic recorded

*Based on evo's [`metrics_interactive`](https://github.com/MichaelGrupp/evo/tree/master/notebooks) notebook for interactive widgets.*

In [ ]:
from __future__ import print_function

from evo.tools import log
log.configure_logging()

In [ ]:
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [ ]:
# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": True, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": True, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
du_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit},
    value=Unit.frames, description='delta_unit'
)
pm_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pr_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

## Load Trajectories

Default setup for [EuRoC](https://projects.asl.ethz.ch/datasets/doku.php?id=kmavvisualinertialdatasets) datasets.

In [ ]:
# Use the CSVs included in the VINS-Mono repo for ground truth
traj_ref_csv = "../../benchmark_publisher/config/MH_01_easy/data.csv"

# Point to ROS bag (with `/vins_estimator/odometry` topic) of VINS-Mono output
traj_est_bag = "anticipated-eurocMH1.bag" # attention and anticipation
# traj_est_bag = "vinsmono-eurocMH1.bag" # no attention and anticipation

In [ ]:
import rosbag
from evo.tools import file_interface
from evo.core import sync
    
# attempt to load output (using algorithm)
bag_handle = rosbag.Bag(traj_est_bag)
traj_est = file_interface.read_bag_trajectory(bag_handle, "/vins_estimator/odometry")

# load ground truth
traj_ref = file_interface.read_euroc_csv_trajectory(traj_ref_csv)

# time sync the trajectories
traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est)

In [ ]:
print(traj_ref)
print(traj_est)

## Absolute Pose Error (APE)

Also known as *Absolute Trajectory Error* (ATE).

In [ ]:
import evo.main_ape as main_ape
import evo.common_ape_rpe as common

count = 0
results = []

def callback_ape(pose_relation, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    result = main_ape.ape(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, align=align, correct_scale=correct_scale)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])

_ = ipywidgets.interact_manual(callback_ape, pose_relation=pr_selector, plot_mode=pm_selector,
                               **{c.description: c.value for c in check_boxes_ape})

## Relative Pose Error (RPE)

Also known as *Relative Trajectory Error* (RTE).

In [ ]:
import evo.main_rpe as main_rpe

count = 0
results = []

def callback_rpe(pose_relation, delta, delta_unit, all_pairs, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="RPE Test #{}".format(count)
    result = main_rpe.rpe(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, delta=delta, delta_unit=delta_unit, 
                          all_pairs=all_pairs, align=align, correct_scale=correct_scale, 
                          support_loop=True)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])

_ = ipywidgets.interact_manual(callback_rpe, pose_relation=pr_selector, plot_mode=pm_selector, 
                               delta=delta_input, delta_unit=du_selector, 
                               **{c.description: c.value for c in check_boxes_rpe})